# Instacart Grocery Basket Analysis (Part 1)

### Contents:
    
1.Importing libraries and files

2.Investigating security implications of the data

3 Categorizing regions

4.Identifying low-activity customers (customers with less than 5 orders) and excluding them from the data.

5.Exporting new data frames

#### 1.Importing libraries and files

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#importing data
path = r'/Users/tiaoshrieh/Desktop/IC Project'

In [3]:
all_merged = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers.pkl'))

#### 2.Investigating security implications of the data

In [4]:
all_merged.head()

,user_id,gender,state,age,date_joined,number_of_dependents,family_status,income,product_id,product_name,...,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,freq_median,order_frequency_flag,region,exclusion_flag
0,26711,Female,Missouri,48,1/1/2017,3,married,165665,196,Soda,...,Busiest day,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,Midwest,high_activity
1,26711,Female,Missouri,48,1/1/2017,3,married,165665,196,Soda,...,Regularly busy,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,Midwest,high_activity
2,26711,Female,Missouri,48,1/1/2017,3,married,165665,196,Soda,...,Busiest day,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,Midwest,high_activity
3,26711,Female,Missouri,48,1/1/2017,3,married,165665,6184,Clementines,...,Regularly busy,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,Midwest,high_activity
4,26711,Female,Missouri,48,1/1/2017,3,married,165665,6184,Clementines,...,Least busy,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,Midwest,high_activity


There is no PII variable in the combined dataset we are working with.
During the process of combining datasets, we used a version of customers dataset without the 'First_Name' and 'Surnam' columns.

#### 3.Categorizing regions

In [5]:
all_merged['state'].value_counts().sort_index()

Alabama                 638003
Alaska                  648495
Arizona                 653964
Arkansas                636144
California              659783
Colorado                639280
Connecticut             623022
Delaware                637024
District of Columbia    613695
Florida                 629027
Georgia                 656389
Hawaii                  632901
Idaho                   607119
Illinois                633024
Indiana                 627282
Iowa                    625493
Kansas                  637538
Kentucky                632490
Louisiana               637482
Maine                   638583
Maryland                626579
Massachusetts           646358
Michigan                630928
Minnesota               647825
Mississippi             632675
Missouri                640732
Montana                 635265
Nebraska                625813
Nevada                  636139
New Hampshire           615378
New Jersey              627692
New Mexico              654494
New York

In [6]:
#Categorizing regions

region1_NE = ['New Hampshire', 'Maine', 'Massachusetts', 'Vermont', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
region2_MW = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana','Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota' ,'Iowa', 'Missouri']
region3_S = ['Delaware', 'Maryland', 'District of Columbia','Virginia','West Virginia','North Carolina','South Carolina','Georgia','Florida','Kentucky','Tennessee','Mississippi','Alabama', 'Oklahoma','Texas','Arkansas','Louisiana']
region4_W = ['Idaho','Montana','Wyoming','Nevada','Utah','Colorado','Arizona','New Mexico', 'Alaska','Washington','Oregon','California','Hawaii']

In [7]:
#Creating 'region' column and assigning values

all_merged.loc[all_merged['state'].isin(region1_NE), 'region'] = 'Northeast'
all_merged.loc[all_merged['state'].isin(region2_MW), 'region'] = 'Midwest'
all_merged.loc[all_merged['state'].isin(region3_S), 'region'] = 'South'
all_merged.loc[all_merged['state'].isin(region4_W), 'region'] = 'West'


In [8]:
all_merged['region'].value_counts()

South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: region, dtype: int64

In [9]:
all_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 33 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   user_id                 int64   
 1   gender                  object  
 2   state                   object  
 3   age                     int64   
 4   date_joined             object  
 5   number_of_dependents    int64   
 6   family_status           object  
 7   income                  int64   
 8   product_id              int64   
 9   product_name            object  
 10  aisle_id                int64   
 11  department_id           int64   
 12  prices                  float64 
 13  order_id                int64   
 14  order_number            int64   
 15  orders_day_of_week      int64   
 16  order_hour_of_day       int64   
 17  days_since_prior_order  float64 
 18  add_to_cart_order       float64 
 19  reordered               float64 
 20  _merge                  category
 21  busies

In [10]:
all_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers.pkl'))

In [11]:
#Using crosstab function to cross the 'region' and 'spending_flag' columns and find the spending habits of customers in each region
Crosstab_regional_spending = pd.crosstab(all_merged['region'], all_merged['spending_flag'], dropna = False)

In [12]:
#customer spending by region
Crosstab_regional_spending

spending_flag,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


#### 4.Identifying low-activity customers (customers with less than 5 orders) and excluding them from the data.

In [13]:
#Creating an exclusion flag
all_merged.loc[all_merged['max_order'] < 5, 'exclusion_flag'] = 'low_activity'
all_merged.loc[all_merged['max_order'] >= 5, 'exclusion_flag'] = 'high_activity'

In [14]:
all_merged['exclusion_flag'].value_counts()

high_activity    30964564
low_activity      1440295
Name: exclusion_flag, dtype: int64

In [15]:
#Creating a subset with the low activity customers excluded

all_merged_loyal_customers = all_merged.loc[all_merged['exclusion_flag'] == 'high_activity']

#### 5.Exporting new data frames

In [18]:
#exporting the subset
all_merged_loyal_customers.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers_loyal.pkl'))